## XBRL API in R 
This sample R code queries the XBRL US API (https://xbrl.us/home/use/xbrl-api/) 
### Enter your XBRL API credentials 
Run the cell below, then enter your XBRL US Web account email, account password, Client ID, and secret in ther input text boxes as they appear on screen, pressing the Enter key on the keyboard after each entry. 

In [ ]:
# load required R libraries

library(httr)
library(jsonlite)
library(tidyverse)

# gather credentials on screen

var.user <- readline("XBRL US Web account email ")
var.pass <- getPass::getPass("Enter the password ")
var.client <- readline("XBRL API Client ID ")
var.secret <- getPass::getPass("XBRL API Secret ")

# authenticate for access token

var.auth <- POST(
     url = "https://api.xbrl.us/oauth2/token",
     body = list(
                 grant_type = "password",
                 platform = "ipynbR",
                 client_id = var.client,
                 client_secret = var.secret,
                 username = var.user,
                 password = var.pass
                 ),
     encode = "form"
     )
var.access <- httr::content(var.auth)$access_token
var.refresh <- httr::content(var.auth)$refresh_token
var.token <- paste("Bearer", var.access)
newaccess = ''
newrefresh = ''

# confirm token authorization print(cat("var.token ", var.token, "\nnewrefresh: ", newrefresh))

accessstatus <- ifelse( var.token =="Bearer ", "There was a problem generating an access token with these credentials. Run this cell again to enter credentials.", "Your access token expires in 60 minutes. After it expires, run the cell immediately below to generate a new token and continue to use the query cell. For now, skip ahead to the section 'Modify the Query'.")

paste0(accessstatus)

In [ ]:
# refresh access token

refresh <- ifelse( newrefresh != '', newrefresh, var.refresh)

var.update <- POST(
     url = "https://api.xbrl.us/oauth2/token",
     body = list(
                 grant_type = "refresh_token",
                 platform = "ipynbR",
                 client_id = var.client,
                 client_secret = var.secret,
                 refresh_token = refresh
                 ),
     encode = "form"
     )

newaccess <- httr::content(var.update)$access_token
newrefresh <- httr::content(var.update)$refresh_token

updatetoken <- paste("Bearer", newaccess)

#confirm token refresh paste0(cat("original access: ", var.access, "\nnew access: ", newaccess, "\nnew refresh: ", newrefresh))

cat(paste0("\n", 'Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cell.'))

### Modify the query 
After the access token is confirmed above, run the cell below to generate results for your account. 

Modify the vaiables in the query below to get new results, _*or*_ comment out - # - the var.url and list(var.data) lines and uncomment the _#alternate >>_ lines in the script, then **re-run the cell below** to change the query.
  
Refer to XBRL API documentation at https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails for other endpoints and parameters to filter and return. 
  
Non-members might not be able to return all data for a query - consider joining XBRL US for comprehensive access - https://xbrl.us/join. 


In [ ]:
# modify query

var.host <- "https://api.xbrl.us/api/v1/"
var.endpoint <- "fact" # fact,report,concept,entity, etc. - some nesting is possible
var.sic_code <- "2080" # Standard Industrial Classification code (beverages)
var.entities <- "0000077476,0000021344,0001083522,0001418135" # SEC Central Index Key values to identify filing entities
var.params <- "&fact.has-dimensions=false&fact.ultimus=true&period.fiscal-period=Y" # endpoint-specific filters
var.concepts <- "Assets,AssetsCurrent,Liabilities,LiabilitiesAndStockholdersEquity,CashCashEquivalentsAndShortTermInvestments" # concepts to look for in filings
var.fields <- "period.fiscal-year.sort(DESC),entity.name.sort(ASC),concept.local-name.sort(ASC),fact.value,fact.decimals,concept.balance-type,concept.is-base,fact.id,dts.id,report.filing-date" # details to be returned

var.url <- paste0(var.host, var.endpoint, "/search?report.sic-code=", var.sic_code, "&concept.local-name=", var.concepts, var.params, "&fields=", var.fields )

#alternate >> var.url <- paste0(var.host, var.endpoint, "/search?entity.cik=", var.entities, "&concept.local-name=", var.concepts, var.params, "&fields=", var.fields)

# request data
accesstoken <- ifelse( newaccess != '', updatetoken, var.token)

var.req <- httr::GET(var.url, httr::add_headers(Authorization = accesstoken))

# parse and display results

options(repr.matrix.max.rows=2000)

var.json <- httr::content(var.req, as = "text")
var.data <- jsonlite::fromJSON(var.json)[[2]]
var.rows <- jsonlite::fromJSON(var.json)[[1]]

cat(paste0("\n\nThe data record limit for this query is ", var.rows$limit, ". ", var.rows$count, " ", var.endpoint, " records were returned for this account, starting with record ", var.rows$offset, ".\n\nNon-members might not be able to return all data for a query - consider XBRL US Membership to improve access - https://xbrl.us/join - and consult documentation for details on using .limit or .offset to manage data record quantity returned. \n\nResults for this query appear below: ", var.url))

list(var.data)

#alternate >> var.data %>% dplyr::filter(str_detect(entity.name, "Pepsi|COCA|JONES|Pepper"))



The data record limit for this query is 2000. 1072 fact records were returned for this account, starting with record 0.

Non-members might not be able to return all data for a query - consider XBRL US Membership to improve access - https://xbrl.us/join - and consult documentation for details on using .limit or .offset to manage data record quantity returned. 

Results for this query appear below: https://api.xbrl.us/api/v1/fact/search?report.sic-code=2080&concept.local-name=Assets,AssetsCurrent,Liabilities,LiabilitiesAndStockholdersEquity,CashCashEquivalentsAndShortTermInvestments&fact.has-dimensions=false&fact.ultimus=true&period.fiscal-period=Y&fields=period.fiscal-year.sort(DESC),entity.name.sort(ASC),concept.local-name.sort(ASC),fact.value,fact.decimals,concept.balance-type,concept.is-base,fact.id,dts.id,report.filing-date

period.fiscal-year,entity.name,concept.local-name,fact.value,fact.decimals,concept.balance-type,concept.is-base,fact.id,dts.id,report.filing-date
2020,Brown-Forman Corporation,Assets,5766000000,-6,1,TRUE,243468007,410785,2020-09-02
2020,Brown-Forman Corporation,AssetsCurrent,3265000000,-6,1,TRUE,243467852,410785,2020-09-02
2020,Brown-Forman Corporation,Liabilities,3791000000,-6,2,TRUE,243468130,410785,2020-09-02
2020,Brown-Forman Corporation,LiabilitiesAndStockholdersEquity,5766000000,-6,2,TRUE,243467924,410785,2020-09-02
2020,"CONSTELLATION BRANDS, INC.",Assets,27323200000,-5,1,TRUE,237189308,396822,2020-07-01
2020,"CONSTELLATION BRANDS, INC.",AssetsCurrent,3484100000,-5,1,TRUE,237188806,396822,2020-07-01
2020,"CONSTELLATION BRANDS, INC.",Liabilities,14848900000,-5,2,TRUE,237188722,396822,2020-07-01
2020,"CONSTELLATION BRANDS, INC.",LiabilitiesAndStockholdersEquity,27323200000,-5,2,TRUE,237188819,396822,2020-07-01
2020,DIAGEO PLC,Assets,33308000000,-6,1,TRUE,240696584,404845,2020-08-07
2020,DIAGEO PLC,Liabilities,24868000000,-6,2,TRUE,240697018,404845,2020-08-07
